In [1]:
import vsnb # importing vsnb automacally registers the ipytho magic
import numpy as np
import time

## Magic: For cells

The `%%monitor` cell magic lets you monitor CPU and memory usage as a cell runs. Just add it at the top of any cell.

In [2]:
%%monitor

for i in range(10):
    result = sum(range(10_000_000))
    time.sleep(0.1)

Use `--persist` or `-p` if you want the widget to persist after cell execution completes. This can be useful as a reminder of CPU/memory used:

In [3]:
%%monitor -p

for i in range(10):
    result = sum(range(10_000_000))
    time.sleep(0.1)

Increase update frequency for smoother visualization use four updates per second:

In [4]:
%%monitor -i 0.25

for i in range(10):
    result = sum(range(10_000_000))
    time.sleep(0.1)

In [5]:
%%monitor -c

for i in range(10):
    result = sum(range(10_000_000))
    time.sleep(0.1)

Produce summary percentages instead of gauges (overrides `-p`):

In [6]:
%%monitor -s

for i in range(10):
    result = sum(range(10_000_000))
    time.sleep(0.1)

CPU %,75,73,65,65,56,48,51,38,32,30


Override automatic computation of gauges per line:

In [7]:
%%monitor -f 2

for i in range(10):
    result = sum(range(10_000_000))
    time.sleep(0.1)

## Context Manager: For scripts or parts of cells

The context manager gives you explicit control over monitoring.
Perfect for Python scripts or when you want programmatic control.

### Basic Context Manager

In [8]:
with vsnb.CPUMonitor():
    for i in range(5):
        result = sum(range(4_000_000))
        time.sleep(0.5)

All arguments to the `monitor` magic can also be passed to `CPUMonitor`:

In [9]:
print("Not monitoring")
for i in range(5):
    result = sum(range(4_000_000))
    time.sleep(0.5)

print("Monitoring")
with vsnb.CPUMonitor(persist=True):
    for i in range(3):
        x = sum(range(1_000_000))
        time.sleep(0.4)
    
print("Not monitoring")
for i in range(5):
    result = sum(range(4_000_000))
    time.sleep(0.5)

Not monitoring
Monitoring


Not monitoring


## Decorator: For Functions

Wrap any function with `@monitor_cpu` to automatically monitor its execution. The decorator takes the same arguments as the magic and the context manager.

In [10]:
@vsnb.monitor(summary=True, color=True)
def some_function():
    for i in range(5):
        result = sum(range(4_000_000))
        time.sleep(0.5)

@vsnb.monitor(summary=True, color=True)
def other_function():
    for i in range(5):
        result = sum(range(4_000_000))
        time.sleep(0.5)        

some_function()
other_function()

CPU %,38,39,47,52,38,33,19,6,4,1


CPU %,42,42,49,53,43,39,20,12,6,4


### Decorator with Custom Settings

In [15]:
@vsnb.monitor(color=True)
def some_function():
    for i in range(5):
        result = sum(range(4_000_000))
        time.sleep(0.5)

some_function()

In [63]:
@ptd.monitor_cpu(update_interval=0.25, show_summary=True)
def process_data(n_iterations):
    """Process data in batches."""
    print(f"Processing {n_iterations} batches...")
    results = []
    for i in range(n_iterations):
        # Simulate batch processing
        batch_result = sum(range(2_000_000))
        results.append(batch_result)
        time.sleep(0.3)
    return results

# Execute
results = process_data(6)
print(f"\nProcessed {len(results)} batches")


Processed 6 batches


## Real Computations <a id="real-computations"></a>

Let's monitor some actual computational work!

### Matrix Operations

In [ ]:
%%monitor

print("Performing large matrix operations...\n")

# Create large matrices
n = 1500
print(f"1. Creating {n}*{n} matrices...")
A = np.random.randn(n, n)
B = np.random.randn(n, n)

print(f"2. Matrix multiplication...")
C = np.dot(A, B)

print(f"3. Computing eigenvalues...")
eigenvalues = np.linalg.eigvals(C[:400, :400])

print(f"4. SVD decomposition...")
U, s, Vh = np.linalg.svd(C[:400, :400], full_matrices=False)

print(f"\nComplete!")
print(f"  Eigenvalues computed: {len(eigenvalues)}")
print(f"  Singular values computed: {len(s)}")

### FFT and Signal Processing

In [ ]:
%%monitor -i 0.3

print("Signal processing tasks...\n")

# Generate signal
print("1. Generating signal...")
t = np.linspace(0, 10, 10_000_000)
signal = np.sin(2 * np.pi * 5 * t) + 0.5 * np.sin(2 * np.pi * 10 * t)

print("2. Computing FFT...")
fft_result = np.fft.fft(signal)

print("3. Computing power spectrum...")
power = np.abs(fft_result)**2

print("4. Finding peaks...")
peaks = power[power > np.percentile(power, 99.9)]

print(f"\nProcessing complete!")
print(f"  Signal length: {len(signal):,}")
print(f"  Peaks found: {len(peaks)}")

### phasic Graph Operations

In [66]:
# Monitor phase-type distribution computations:

# First, create a phase-type distribution
print("Building phase-type distribution...")

g = ptd.Graph(1)
start = g.starting_vertex()

# Create a 5-phase Erlang-like distribution
vertices = []
for i in range(5):
    v = g.find_or_create_vertex([i])
    vertices.append(v)

# Connect phases
start.add_edge(vertices[0], 1.0)
for i in range(4):
    vertices[i].add_edge(vertices[i+1], 2.0 + i * 0.5)

g.normalize()

print(f"  Graph created with {g.vertices_length()} vertices")
print(f"  Ready for computation!")

Building phase-type distribution...
  Graph created with 6 vertices
  Ready for computation!


In [ ]:
%%monitor

print("Computing phase-type distribution PDF...\n")

# Evaluate PDF at many time points
print("1. Generating time points...")
times = np.linspace(0.01, 10.0, 50_000)

print("2. Computing PDF values...")
pdf_values = g.pdf_batch(times)

print("3. Analyzing results...")
max_pdf = np.max(pdf_values)
argmax = times[np.argmax(pdf_values)]
total_prob = np.trapezoid(pdf_values, times)

print(f"\n  PDF computation complete!")
print(f"  Time points: {len(times):,}")
print(f"  Max PDF: {max_pdf:.4f} at t={argmax:.2f}")
print(f"  Total probability: {total_prob:.4f}")

### Batch Moments Calculation

In [ ]:
%%monitor

print("Computing distribution moments...\n")

# Compute moments 1 through 10
moment_orders = np.arange(1, 11)
print(f"Computing moments {moment_orders[0]} through {moment_orders[-1]}...")

moments = g.moments_batch(moment_orders)

# Calculate statistics
mean = moments[0]
variance = moments[1] - mean**2
std_dev = np.sqrt(variance)

print(f"\n  Moments computed!")
print(f"  Mean: {mean:.4f}")
print(f"  Variance: {variance:.4f}")
print(f"  Std Dev: {std_dev:.4f}")
print(f"\n  First 5 moments:")
for i in range(5):
    print(f"    E[T^{i+1}] = {moments[i]:.4f}")

## Advanced Configuration <a id="advanced"></a>

### Comparing Different Workloads

In [69]:
def workload_light():
    """Light CPU usage."""
    for _ in range(5):
        x = sum(range(500_000))
        time.sleep(0.5)

def workload_heavy():
    """Heavy CPU usage."""
    for _ in range(5):
        x = sum(range(5_000_000))
        time.sleep(0.1)

print("=" * 60)
print("LIGHT WORKLOAD")
print("=" * 60)
with ptd.CPUMonitor():
    workload_light()

print("\n" + "=" * 60)
print("HEAVY WORKLOAD")
print("=" * 60)
with ptd.CPUMonitor():
    workload_heavy()

print("\n  Comparison complete!")


  Comparison complete!


### Monitoring Multiple Phases

In [ ]:
%%monitor

phases = [
    ("Data Loading", 1_000_000, 3),
    ("Preprocessing", 2_000_000, 3),
    ("Training", 4_000_000, 4),
    ("Validation", 1_500_000, 2),
]

print("Multi-phase computation...\n")

for phase_name, workload, iterations in phases:
    print(f"Phase: {phase_name}")
    for i in range(iterations):
        result = sum(range(workload))
        time.sleep(0.3)
    print(f"  {phase_name} complete\n")

print("\nAll phases complete!")

## SLURM usage

### Check Current Environment

In [71]:
import os

print("Current Environment:")
print("=" * 60)

# Check if we're on SLURM
if 'SLURM_JOB_ID' in os.environ:
    print(" Running on SLURM")
    print(f"  Job ID: {os.environ.get('SLURM_JOB_ID')}")
    print(f"  Nodes: {os.environ.get('SLURM_JOB_NUM_NODES', 'N/A')}")
    print(f"  CPUs per task: {os.environ.get('SLURM_CPUS_PER_TASK', 'N/A')}")
    print(f"  Node list: {os.environ.get('SLURM_JOB_NODELIST', 'N/A')}")
else:
    print(" Running locally (not on SLURM)")

# Show detected nodes
print(f"\nDetected Resources:")
nodes = ptd.detect_compute_nodes()
for node in nodes:
    print(f"  • {node.name}: {node.cpu_count} CPUs")
    if node.allocated_cpus:
        print(f"    Allocated cores: {len(node.allocated_cpus)}")

Current Environment:
 Running locally (not on SLURM)

Detected Resources:
  • perry: 10 CPUs
    Allocated cores: 10


## Tips and Tricks <a id="tips"></a>

### Quick Testing Without Summary

In [72]:
# For quick tests, disable summary
with ptd.CPUMonitor(show_summary=False):
    result = sum(range(3_000_000))

print(f"Quick test complete: {result}")

Quick test complete: 4499998500000


### Combine with Parallel Configuration

In [73]:
# Initialize parallel configuration
try:
    config = ptd.init_parallel(cpus=4)
    print(f"Parallel config: {config.device_count} devices")
    print(f"  Strategy: {config.strategy}")
except Exception as e:
    print(f"Could not initialize parallel: {e}")

# Now monitor with parallel execution
with ptd.CPUMonitor():
    times = np.linspace(0.1, 5.0, 10_000)
    # This will use parallel configuration if available
    try:
        pdf = g.pdf_batch(times)
        print(f"Computed {len(pdf)} PDF values")
    except Exception as e:
        print(f"Computation not available: {e}")

### Adjust Update Interval Based on Workload

In [74]:
print("Fast computation → frequent updates:")
with ptd.CPUMonitor(update_interval=0.2):
    for _ in range(5):
        x = sum(range(1_000_000))
        time.sleep(0.3)

print("\nSlow computation → less frequent updates:")
with ptd.CPUMonitor(update_interval=1.0):
    for _ in range(3):
        x = sum(range(5_000_000))
        time.sleep(1.0)

print("\nBoth complete!")


Both complete!


### Monitor Specific Functions Only

In [75]:
def preprocessing():
    """Not monitored."""
    print("Preprocessing (not monitored)...")
    time.sleep(0.5)

@ptd.monitor_cpu(show_summary=False)
def expensive_computation():
    """Monitored."""
    print("Expensive computation (monitored)...")
    for _ in range(4):
        x = sum(range(3_000_000))
        time.sleep(0.4)

def postprocessing():
    """Not monitored."""
    print("Postprocessing (not monitored)...")
    time.sleep(0.5)

# Run pipeline
preprocessing()
expensive_computation()
postprocessing()

print("\nPipeline complete!")

Postprocessing (not monitored)...

Pipeline complete!
